In [23]:
import pandas as pd
import datetime as dt

In [27]:
# EDA for spot data
price_source = "rbi"
ccy = "EURUSD"
ccy_prices = pd.read_csv(f"data/{price_source}/{ccy}.csv")

In [28]:
ccy_prices.describe(include="all")

,timestamp,Bid,Ask
count,3379,2296.000000,2329.000000
unique,3379,NaN,NaN
top,2022-02-28 21:00:00.055633+00:00,NaN,NaN
freq,1,NaN,NaN
mean,NaN,1.029816,1.031333
std,NaN,0.047628,0.047772
min,NaN,0.954240,0.954880
25%,NaN,0.992160,0.993090
50%,NaN,1.014480,1.016220
75%,NaN,1.062682,1.071990


In [98]:
# EDA for options data
month = 3
year = 2022

if month < 10:
    month = '0' + str(month)
option_prices = pd.read_csv(f"data/forts/{year}{month}_opt_deal.csv")
option_prices.columns

Index(['#SYMBOL', 'SYSTEM', 'MOMENT', 'ID_DEAL', 'PRICE_DEAL', 'VOLUME',
       'OPEN_POS', 'DIRECTION'],
      dtype='object')

In [99]:
option_prices = option_prices.drop("ID_DEAL", axis="columns")

In [100]:
option_prices["MOMENT"] = option_prices["MOMENT"].apply(lambda x: dt.date(year=int(str(x)[:4]), month=int(str(x)[4:6]), day=int(str(x)[6:8])))

In [101]:
option_prices["STRIKE"] = option_prices["#SYMBOL"].apply(lambda x: float(x[2:7]) / 1000 if x[0:2] == "Si" else None)

In [102]:
option_prices.head()

,#SYMBOL,SYSTEM,MOMENT,PRICE_DEAL,VOLUME,OPEN_POS,DIRECTION,STRIKE
0,Si90000BC2,C,2022-03-02,4399.0,1,42030,B,90.0
1,Si95000BC2,C,2022-03-02,3600.0,2,6098,B,95.0
2,Si95000BC2,C,2022-03-02,3600.0,3,6092,B,95.0
3,Si70000BO2,P,2022-03-02,22.0,3,12868,S,70.0
4,Si70000BO2,P,2022-03-02,22.0,3,12868,S,70.0


In [103]:
option_prices.describe(include="all")

,#SYMBOL,SYSTEM,MOMENT,PRICE_DEAL,VOLUME,OPEN_POS,DIRECTION,STRIKE
count,86587,86587,86587,86587.000000,86587.000000,86587.000000,86587,60759.000000
unique,2826,2,20,NaN,NaN,NaN,2,NaN
top,Si94500BC2,C,2022-03-29,NaN,NaN,NaN,B,NaN
freq,2347,52819,12273,NaN,NaN,NaN,48207,NaN
mean,NaN,NaN,NaN,6081.733969,7.894476,8292.815319,NaN,58.436193
std,NaN,NaN,NaN,10768.310311,250.689455,20369.026585,NaN,37.203411
min,NaN,NaN,NaN,0.000100,1.000000,0.000000,NaN,10.000000
25%,NaN,NaN,NaN,70.000000,1.000000,146.000000,NaN,11.000000
50%,NaN,NaN,NaN,1240.000000,1.000000,664.000000,NaN,80.000000
75%,NaN,NaN,NaN,5900.000000,4.000000,3650.000000,NaN,90.000000


In [104]:
# Calculate average moneyness for USDRUB options
from src.working_with_files.preprocessing import get_option_prices, get_asset_prices

PATH = "data/"

asset = "USDRUB"

moex_spot = get_asset_prices(path=PATH, asset=asset, price_source="moex", 
                             delta_seconds=60 * 60)
# moex_spot

# Leave only the last value => closing value for the spot (rough estimation)
out_moex_spot = {}
for k, v in moex_spot.items():
    out_moex_spot[dt.datetime.date(k)] = v
out_moex_spot

{datetime.date(2022, 3, 1): 99.70244749999999,
 datetime.date(2022, 3, 2): 106.01474,
 datetime.date(2022, 3, 3): 105.775175,
 datetime.date(2022, 3, 4): 107.9737475,
 datetime.date(2022, 3, 9): 119.322365,
 datetime.date(2022, 3, 10): 117.03959,
 datetime.date(2022, 3, 11): 117.2112375,
 datetime.date(2022, 3, 14): 113.51015,
 datetime.date(2022, 3, 15): 109.75394,
 datetime.date(2022, 3, 16): 108.302195,
 datetime.date(2022, 3, 17): 104.610125,
 datetime.date(2022, 3, 18): 104.32458,
 datetime.date(2022, 3, 21): 104.212265,
 datetime.date(2022, 3, 22): 103.6224775,
 datetime.date(2022, 3, 23): 98.91548,
 datetime.date(2022, 3, 24): 96.2430225,
 datetime.date(2022, 3, 25): 95.9066175,
 datetime.date(2022, 3, 28): 90.0018075,
 datetime.date(2022, 3, 29): 84.74993,
 datetime.date(2022, 3, 30): 84.2431125,
 datetime.date(2022, 3, 31): 83.38395,
 datetime.date(2022, 4, 1): 83.6610975,
 datetime.date(2022, 4, 4): 83.519895,
 datetime.date(2022, 4, 5): 83.5531075,
 datetime.date(2022, 4, 6)

In [105]:
option_prices["% ATM"] = option_prices.apply(lambda row: float(row.STRIKE) / float(out_moex_spot[row.MOMENT]), axis=1)

In [106]:
option_prices.head()

,#SYMBOL,SYSTEM,MOMENT,PRICE_DEAL,VOLUME,OPEN_POS,DIRECTION,STRIKE,% ATM
0,Si90000BC2,C,2022-03-02,4399.0,1,42030,B,90.0,0.848939
1,Si95000BC2,C,2022-03-02,3600.0,2,6098,B,95.0,0.896102
2,Si95000BC2,C,2022-03-02,3600.0,3,6092,B,95.0,0.896102
3,Si70000BO2,P,2022-03-02,22.0,3,12868,S,70.0,0.660286
4,Si70000BO2,P,2022-03-02,22.0,3,12868,S,70.0,0.660286
